## VGG16 Model

In [3]:
# load train and test dataset

def load_cifar_dataset():
    # load dataset
    (trainX, trainY), (testX, testY) = cifar10.load_data()
    
    # one hot encode target values
    trainY = to_categorical(trainY)
    testY = to_categorical(testY)
    return trainX, trainY, testX, testY

def load_server_dataset(folder_name):
    '''
    Given a folder name on Materiall's shared server, read in all the files, convert them to pixels, 
    and separate them into train and testing data.
    '''
    

Model fitted! Epochs=100, Batch Size=64


In [83]:
import sys
import matplotlib.pyplot as plt
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical

def prep_pixels(train, test):
    train_norm = train.astype('float32') / 255.0
    test_norm = test.astype('float32') / 255.0
    return train_norm, test_norm
 
# plot diagnostic learning curves
def summarize_diagnostics(history):
    # plot loss
    plt.subplot(211)
    plt.title('Cross Entropy Loss')
    plt.plot(history.history['loss'], color='blue', label='train')
    plt.plot(history.history['val_loss'], color='orange', label='test')
    # plot accuracy
    plt.subplot(212)
    plt.title('Classification Accuracy')
    plt.plot(history.history['accuracy'], color='blue', label='train')
    plt.plot(history.history['val_accuracy'], color='orange', label='test')
    # save plot to file
    plt.savefig('../images/vgg16_diagnostic_plot.png')
    plt.close()
    
# run the test harness for evaluating a model
def run_test_harness(load_dataset, epochs=100, batch_size=64, verbose=0):
    # load dataset
    trainX, trainY, testX, testY = load_dataset()
    print('Dataset Loaded!')
    # prepare pixel data
    trainX, testX = prep_pixels(trainX, testX)
    print('Dataset Preprocessed!')
    # define model
    print('Defining Model...')
    model = define_model()
    # fit model
    print('Fitting model...')
    history = model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, validation_data=(testX, testY), verbose=verbose)
    print('Model fitted! Epochs=%d, Batch Size=%d' % (epochs, batch_size))
    # evaluate model
    _, acc = model.evaluate(testX, testY, verbose=0)
    print('Model Evalution:')
    print('> %.3f' % (acc * 100.0))
    # learning curves
    summarize_diagnostics(history)
    
    return model

In [87]:
# define cnn model
def define_model():

    # grab the pre-trained VGG16 model, removing the top layers and changing the input shape
    model = VGG16(weights="imagenet", include_top=False, input_shape=(32, 32, 3))
        
    # freeze pre-trained layers
    for layer in model.layers:
        layer.trainable = False

    # add new classifier layers
    flat1 = Flatten()(model.layers[-1].output)
    class1 = Dense(1024, activation='relu')(flat1)
    output = Dense(10, activation='softmax')(class1)

    # define new model with top layers
    model = Model(inputs=model.inputs, outputs=output)
    opt = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    
    # for output while training
    model.summary()

    return model